In [2]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
from matplotlib import rcParams
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
# 한글 폰트 설정
rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕으로 설정
rcParams['axes.unicode_minus'] = False     # 음수 기호 문제 해결

In [3]:
# EXCEL파일 읽기
file_path = './Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후.xlsx'
df_bigkinds = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [4]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,URL,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사
0,02100101.20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...
1,02100101.20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...
2,02100311.20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,http://www.sedaily.com/NewsView/1OKUY3ELL1,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...
3,02100101.20170902162415001,2017-09-02,매일경제,디지털뉴스국,서울 도심 곳곳서 태극기 집회,정치>청와대,정치>행정_자치,정치>선거,사회>사회갈등>시위,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"서울 도심 곳곳서 태극기 집회 서울,도심,태극기,집회,주말,도심,서울,도심,박근혜,...",서울 도심 곳곳서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 돌 시ㅁ 곳곳 서 태극기 집회 서울 도 시ㅁ 태극기 집회 주말 도 시ㅁ 서울 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...
4,02100101.20170904091746001,2017-09-04,매일경제,고득관,"코스피, 북핵 리스크에 2330선 `털썩`",정치>북한,정치>청와대,NaN,범죄>기업범죄>거래제한,재해>자연재해>지진,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"코스피, 북핵 리스크에 2330선 `털썩` 코스피,리스크,2330선,북한,지정학,리...",코스피 북핵 리스크에 2330선 털썩 코스피 리스크 2330선 북한 지정학 리스크 ...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북 ㄴ 지정학...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북한 지정학 ...,코스피 북핵 리스크 2330 코스피 리스크 2330 북한 지정학 리스크 고조 코스피...,코스 피 북 핵 리스크 선 코스 피 리스크 선 북한 지정학 리스크 고조 코스 피 급...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,02100101.20240829144509001,2024-08-29,매일경제,이호준 매경이코노미 인턴기자(lhj0756@naver.com),재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구?,정치>국회_정당,정치>선거,NaN,NaN,NaN,...,http://www.mk.co.kr/article/11104605,NaN,"재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구? 재산,페라리,소유,...",재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구 재산 페라리 소유 국...,재산 333억원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의원 ...,재산 333억원 페라리 소유 국회의원 재산신고 1위 재산 페라리 소유 국회의원 재산...,재산 333 억 원 페라리 소유 국회 의원 재산 신고 1 위 누 재산 페라리 소유 ...,재산 333 억 원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의...,재산 333 페라리 소유 국회 의원 재산 신고 재산 페라리 소유 국회 의원 재산 신...,재산 원 페라리 소유 국회의원 재산 신고 위 재산 페라리 소유 국회의원 재산 신고 ...
5408,02100311.20240829000230001,2024-08-29,서울경제,김예솔 기자,신규 국회의원 평균재산 27억 고동진 333억 '최고',정치>국회_정당,정치>선거,NaN,NaN,NaN,...,http://www.sedaily.com/NewsView/2DD7CNUYYS,NaN,"신규 국회의원 평균재산 27억 고동진 333억 '최고' 신규,국회의원,평균,재산,고...",신규 국회의원 평균재산 

In [5]:
# KoBERT 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("kwoncho/KoFinBERT")
model = AutoModelForSequenceClassification.from_pretrained("kwoncho/KoFinBERT")

In [6]:
text_classifier = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    truncation=True,  # 긴 텍스트를 자동으로 자름
    max_length=512,   # 최대 길이를 512로 제한
    top_k=None,       # 모든 레이블의 확률 반환
    device=0          # GPU 사용 가능 시 (0), 그렇지 않으면 CPU (-1)
)

In [7]:
max_length = 512

s = time.time()
predicted_labels = []
predicted_scores = []
# tqdm을 사용하여 처리 상태 표시
for text in tqdm(df_bigkinds['제목_키워드_전처리후_Hannanum'], desc="Predicting"):
    # 감정 분석 수행
    prediction = text_classifier(text)
    
    # 레이블을 숫자 값으로 매핑
    label_mapping = {
        'positive': 1,
        'neutral': 0,
        'negative': -1
    }
    
    # 연속적인 점수 직접 계산
    score = 0
    for item in prediction[0]:
        label = item['label']
        prob = item['score']
        score += label_mapping[label] * prob
    predicted_scores.append(score)

print("Prediction Time: ", time.time() - s)

Predicting: 100%|██████████████████████████████████████████████████████████████████| 5412/5412 [01:39<00:00, 54.38it/s]

Prediction Time:  99.51894187927246


In [13]:
# 뉴스 식별자 컬럼에서 '.' 이후의 값만 조회
df_bigkinds['뉴스 식별자'] = df_bigkinds['뉴스 식별자'].str.split('.').str[1]

In [14]:
# 데이터프레임에 결과 추가
df_bigkinds['KoFinBERT_Sentiment'] = predicted_scores
# 소수점 1자리로 반올림
df_bigkinds['KoFinBERT_Sentiment'] = df_bigkinds['KoFinBERT_Sentiment'].round(1)
df_bigkinds['KoFinBERT_Sentiment'] = df_bigkinds['KoFinBERT_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,KoFinBERT_Sentiment
0,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,0.0
1,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,-0.1
2,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,-0.9
3,20170902162415001,2017-09-02,매일경제,디지털뉴스국,서울 도심 곳곳서 태극기 집회,정치>청와대,정치>행정_자치,정치>선거,사회>사회갈등>시위,NaN,...,NaN,"서울 도심 곳곳서 태극기 집회 서울,도심,태극기,집회,주말,도심,서울,도심,박근혜,...",서울 도심 곳곳서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 돌 시ㅁ 곳곳 서 태극기 집회 서울 도 시ㅁ 태극기 집회 주말 도 시ㅁ 서울 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,0.0
4,20170904091746001,2017-09-04,매일경제,고득관,"코스피, 북핵 리스크에 2330선 `털썩`",정치>북한,정치>청와대,NaN,범죄>기업범죄>거래제한,재해>자연재해>지진,...,NaN,"코스피, 북핵 리스크에 2330선 `털썩` 코스피,리스크,2330선,북한,지정학,리...",코스피 북핵 리스크에 2330선 털썩 코스피 리스크 2330선 북한 지정학 리스크 ...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북 ㄴ 지정학...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북한 지정학 ...,코스피 북핵 리스크 2330 코스피 리스크 2330 북한 지정학 리스크 고조 코스피...,코스 피 북 핵 리스크 선 코스 피 리스크 선 북한 지정학 리스크 고조 코스 피 급...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,20240829144509001,2024-08-29,매일경제,이호준 매경이코노미 인턴기자(lhj0756@naver.com),재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구?,정치>국회_정당,정치>선거,NaN,NaN,NaN,...,NaN,"재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구? 재산,페라리,소유,...",재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구 재산 페라리 소유 국...,재산 333억원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의원 ...,재산 333억원 페라리 소유 국회의원 재산신고 1위 재산 페라리 소유 국회의원 재산...,재산 333 억 원 페라리 소유 국회 의원 재산 신고 1 위 누 재산 페라리 소유 ...,재산 333 억 원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의...,재산 333 페라리 소유 국회 의원 재산 신고 재산 페라리 소유 국회 의원 재산 신...,재산 원 페라리 소유 국회의원 재산 신고 위 재산 페라리 소유 국회의원 재산 신고 ...,0.0
5408,20240829000230001,2024-08-29,서울경제,김예솔 기자,신규 국회의원 평균재산 27억 고동진 333억 '최고',정치>국회_정당,정치>선거,NaN,NaN,NaN,...,NaN,"신규 국회의원 평균재산 27억 고동진 333억 '최고' 신규,국회의원,평균,재산,고...",신규 국회의원 평균재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동진...,신규 국회의원 평균 재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동...,신규 국회의원 평균재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동진...,신규 국회의원 평균 재산 27 억 고동진 333 억 최고 신규 국회의원 평균 재산 ...,신규 국회의원 평균 재산 27 억 고동진 333 억 최고 신규 국회의원 평균 재산 ...,신규 국회의원 평균 재산 27 고동진 333 최고 신규 국회의원 평균 재산 고동진 ...,신규 국회의원 평균 재산 고동진 최고 신규 국회의원 평균 재산 고동진 최고 조국 이...,0.0
540

전체 분석 후 분석제외 대상과 중복건 제거

In [15]:
df_bigkinds = df_bigkinds[df_bigkinds['분석제외 여부'].isnull()]

In [16]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,KoFinBERT_Sentiment
0,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,0.0
1,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,-0.1
2,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,-0.9
3,20170902162415001,2017-09-02,매일경제,디지털뉴스국,서울 도심 곳곳서 태극기 집회,정치>청와대,정치>행정_자치,정치>선거,사회>사회갈등>시위,NaN,...,NaN,"서울 도심 곳곳서 태극기 집회 서울,도심,태극기,집회,주말,도심,서울,도심,박근혜,...",서울 도심 곳곳서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 돌 시ㅁ 곳곳 서 태극기 집회 서울 도 시ㅁ 태극기 집회 주말 도 시ㅁ 서울 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근 ...,서울 도심 곳곳 서 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,서울 도심 곳곳 태극기 집회 서울 도심 태극기 집회 주말 도심 서울 도심 박근혜 단...,0.0
4,20170904091746001,2017-09-04,매일경제,고득관,"코스피, 북핵 리스크에 2330선 `털썩`",정치>북한,정치>청와대,NaN,범죄>기업범죄>거래제한,재해>자연재해>지진,...,NaN,"코스피, 북핵 리스크에 2330선 `털썩` 코스피,리스크,2330선,북한,지정학,리...",코스피 북핵 리스크에 2330선 털썩 코스피 리스크 2330선 북한 지정학 리스크 ...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북 ㄴ 지정학...,코스 피 북 핵 리스크 2330 선 털썩 코스 피 리스크 2330 선 북한 지정학 ...,코스피 북핵 리스크 2330 코스피 리스크 2330 북한 지정학 리스크 고조 코스피...,코스 피 북 핵 리스크 선 코스 피 리스크 선 북한 지정학 리스크 고조 코스 피 급...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,20240829144509001,2024-08-29,매일경제,이호준 매경이코노미 인턴기자(lhj0756@naver.com),재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구?,정치>국회_정당,정치>선거,NaN,NaN,NaN,...,NaN,"재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구? 재산,페라리,소유,...",재산 333억원에 페라리 소유...국회의원 재산신고 1위 누구 재산 페라리 소유 국...,재산 333억원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의원 ...,재산 333억원 페라리 소유 국회의원 재산신고 1위 재산 페라리 소유 국회의원 재산...,재산 333 억 원 페라리 소유 국회 의원 재산 신고 1 위 누 재산 페라리 소유 ...,재산 333 억 원 페라리 소유 국회의원 재산 신고 1 위 재산 페라리 소유 국회의...,재산 333 페라리 소유 국회 의원 재산 신고 재산 페라리 소유 국회 의원 재산 신...,재산 원 페라리 소유 국회의원 재산 신고 위 재산 페라리 소유 국회의원 재산 신고 ...,0.0
5408,20240829000230001,2024-08-29,서울경제,김예솔 기자,신규 국회의원 평균재산 27억 고동진 333억 '최고',정치>국회_정당,정치>선거,NaN,NaN,NaN,...,NaN,"신규 국회의원 평균재산 27억 고동진 333억 '최고' 신규,국회의원,평균,재산,고...",신규 국회의원 평균재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동진...,신규 국회의원 평균 재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동...,신규 국회의원 평균재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동진...,신규 국회의원 평균 재산 27 억 고동진 333 억 최고 신규 국회의원 평균 재산 ...,신규 국회의원 평균 재산 27 억 고동진 333 억 최고 신규 국회의원 평균 재산 ...,신규 국회의원 평균 재산 27 고동진 333 최고 신규 국회의원 평균 재산 고동진 ...,신규 국회의원 평균 재산 고동진 최고 신규 국회의원 평균 재산 고동진 최고 조국 이...,0.0
540

뉴스 식별자 최소갑 기준으로 제목_키워드_전처리후 그룹핑

In [17]:
# 그룹핑할 컬럼 리스트
group_columns = ['제목_키워드_전처리후_Hannanum']

# 각 그룹에서 뉴스 식별자 컬럼의 최소값을 유지
df_bigkinds2 = df_bigkinds.groupby(group_columns, as_index=False).agg({'뉴스 식별자': 'min'})

In [18]:
df_bigkinds2

,제목_키워드_전처리후_Hannanum,뉴스 식별자
0,- 관계 개 선 수혜 강세 화장품 여행 매수 기회 테마 섹터 수혜 개 선 강세 화장...,20171129112950001
1,- 만나 ㄹ까 리룡남 부총리 나이지리아 방문길 베이징 경유 리룡남 부총리 나이지리아...,20190608162001001
2,- 바 든 만찬 10대그룹 총수 총출동 만찬 10대 그룹 총수 21일 국립중앙박물관...,20220518171014001
3,- 바 든 면담계획 없 대북특사론 도 수 면 아래 면담계획 대북특사론 수 면 아래 ...,20220519095617001
4,- 바 든 삼성 차세대 반도체 기지 서 공동 연설 칩4 동맹 현실 화하 바 든 삼성...,20220518173657001
...,...,...
4945,효성티앤씨 대표이사 김치형 부사장 내정 효성티앤씨 대표 이사 내정 김치 부사장 효성...,20220112141736001
4946,후보 누군지 잘 모르 요 유권자 평균 34세 ㄴ 경기 도 마을 아직 은 낯설 ㄴ 3...,20240314154003001
4947,흑석 김의겸 매입 었다는 서초 우면동 투 가치 는 흑석 김의겸 매입 서초 우면동 투...,20210702153458002
4948,흙수저 의원들 좋 은 일자리 기회 사다리 흙수저 의원 일자리 기회 사다리 광주여상 ...,20201007175758001


In [19]:
# 필요한 컬럼만 선택하여 두 데이터프레임 병합
big_kinds_sentiment = pd.merge(
    df_bigkinds[['일자','제목_키워드_전처리후_Hannanum', '뉴스 식별자','KoFinBERT_Sentiment']],
    df_bigkinds2[['제목_키워드_전처리후_Hannanum', '뉴스 식별자']],
    on=['제목_키워드_전처리후_Hannanum', '뉴스 식별자'],
    how='inner'
)
big_kinds_sentiment = big_kinds_sentiment[['뉴스 식별자','일자','제목_키워드_전처리후_Hannanum','KoFinBERT_Sentiment']]
big_kinds_sentiment

,뉴스 식별자,일자,제목_키워드_전처리후_Hannanum,KoFinBERT_Sentiment
0,20170901174354001,2017-09-01,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,0.0
1,20170901141935001,2017-09-01,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,-0.1
2,20170901153936001,2017-09-01,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,-0.9
3,20170902162415001,2017-09-02,서울 돌 시ㅁ 곳곳 서 태극기 집회 서울 도 시ㅁ 태극기 집회 주말 도 시ㅁ 서울 ...,0.0
4,20170904091746001,2017-09-04,코스피 북핵 리스크 2330 선 털썩 코스피 리스크 2330 선 북한 지정학 리스크...,0.0
...,...,...,...,...
4945,20240829144509001,2024-08-29,재산 333억원 페라리 소유 국회의원 재산신고 1위 재산 페라리 소유 국회의원 재산...,0.0
4946,20240829000230001,2024-08-29,신규 국회의원 평균재산 27억 고동진 333억 최고 신규 국회의원 평균 재산 고동진...,0.0
4947,20240829070127001,2024-08-29,22대 국회 신규등록 최고 부자 는 333억 고동진 조국 52억 이준석 12억 부 ...,0.0
4948,20240829002204001,2024-08-29,333억 고동진 22대 초선 재산 1위 333억 고동진 초선 재산 국회 입성 국회의...,0.0


In [20]:
big_kinds_sentiment.to_excel('./Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Hannanum_KoFinBERT_id.xlsx', index=False)

# 아래코드는 마지막 데이터 미생성으로 인해 생성한 코드로 위의 과정이 오래 걸려 추가
file_path = './Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Hannanum_KoFinBERT_id.xlsx'
big_kinds_sentiment = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [5]:
big_kinds_sentiment_daily = big_kinds_sentiment.groupby('일자', as_index=False)['KoFinBERT_Sentiment'].mean()
# 소수점 1자리로 반올림
big_kinds_sentiment_daily['KoFinBERT_Sentiment'] = big_kinds_sentiment_daily['KoFinBERT_Sentiment'].round(1)
big_kinds_sentiment_daily['KoFinBERT_Sentiment'] = big_kinds_sentiment_daily['KoFinBERT_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
big_kinds_sentiment_daily

,일자,KoFinBERT_Sentiment
0,2017-09-01,-0.3
1,2017-09-02,0.0
2,2017-09-04,0.0
3,2017-09-05,0.0
4,2017-09-06,0.0
...,...,...
1648,2024-08-23,0.0
1649,2024-08-24,0.0
1650,2024-08-26,0.0
1651,2024-08-27,0.0


In [6]:
big_kinds_sentiment_daily = big_kinds_sentiment_daily.rename(columns={'일자': 'date', 'KoFinBERT_Sentiment': 'sentiment'})

In [10]:
big_kinds_sentiment_daily

,date,sentiment
0,2017-09-01,-0.3
1,2017-09-02,0.0
2,2017-09-04,0.0
3,2017-09-05,0.0
4,2017-09-06,0.0
...,...,...
1648,2024-08-23,0.0
1649,2024-08-24,0.0
1650,2024-08-26,0.0
1651,2024-08-27,0.0


In [12]:
big_kinds_sentiment_daily.to_excel('./Text_Data/bigkinds/삼성전자_정치_20170901-20240830_전처리후_Hannanum_KoFinBERT_id_daily.xlsx', index=False)